In [5]:
# Standard library imports
import sys
from pathlib import Path
from typing import Tuple, Optional

# Third-party imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import hilbert

# Add src to path for local imports
src_path = Path.cwd().parent.parent.parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Local imports
from signals import generate_sine_wave
from spectral import compute_psd_welch
from filtering import bandpass_filter, lowpass_filter
from colors import COLORS
from constants import EEG_BANDS, BAND_COLORS

# Define color constants
PRIMARY_BLUE = COLORS["signal_1"]
SECONDARY_PINK = COLORS["signal_2"]
PRIMARY_GREEN = COLORS["signal_3"]
ACCENT_GOLD = COLORS["signal_4"]
ACCENT_PURPLE = COLORS["signal_5"]
ACCENT_TEAL = COLORS["signal_6"]
PRIMARY_RED = COLORS["negative"]
PLOT_BG_COLOR = COLORS["background"]

# Matplotlib configuration
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 11

print(f"NumPy version: {np.__version__}")
print(f"Source path: {src_path}")

NumPy version: 2.3.5
Source path: /Users/remyramadour/Workspace/PPSP/Workshops/ConnectivityMetricsTutorials/src


---

## 1. Introduction

Many connectivity metrics in hyperscanning research require **phase** or **amplitude** information:

- **Phase-based metrics** (PLV, PLI, wPLI): Measure synchronization by comparing phases between signals
- **Amplitude-based metrics** (Envelope Correlation, Power Correlation): Track co-fluctuations in signal strength

But there's a fundamental problem: **a raw signal doesn't have a single "phase."** A real EEG signal is a mixture of many frequencies, each with its own phase. Which phase do we report?

The solution involves two steps:
1. **Band-pass filter** the signal to isolate a narrow frequency range (e.g., alpha 8-13 Hz)
2. Apply the **Hilbert transform** to extract instantaneous amplitude and phase

The Hilbert transform is the mathematical tool that enables this extraction. It's the foundation for virtually all phase-based and many amplitude-based connectivity analyses in neuroscience.

---

## 2. The Problem — What is "Phase" of a Complex Signal?

For a **pure sine wave**, phase is well-defined at every time point:

$$x(t) = A \sin(2\pi f t + \phi)$$

At any time $t$, the phase is simply $2\pi f t + \phi$.

But for a **composite signal** (sum of multiple frequencies), there is no single phase. Which frequency's phase do we report?

Let's visualize this problem:

In [6]:
# ============================================================================
# VISUALIZATION 1: Phase Ambiguity in Composite Signals
# ============================================================================

# Parameters
fs = 250  # Sampling rate
duration = 1.0  # seconds
t = np.arange(0, duration, 1/fs)

# Create signals
pure_10hz = np.sin(2 * np.pi * 10 * t)
composite = (np.sin(2 * np.pi * 5 * t) + 
             np.sin(2 * np.pi * 10 * t) + 
             0.5 * np.sin(2 * np.pi * 20 * t))

# Band-pass filter to isolate 10 Hz
filtered_10hz = bandpass_filter(composite, 8, 12, fs=fs)

# Create figure
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Plot 1: Pure sine wave
ax1 = axes[0]
ax1.plot(t * 1000, pure_10hz, color=PRIMARY_BLUE, linewidth=2)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Pure 10 Hz Sine Wave — Phase is Well-Defined', fontsize=12, fontweight='bold')
ax1.set_xlim(0, 500)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)

# Add phase annotation
ax1.annotate('Phase = 0°', xy=(0, 0), xytext=(30, 0.7),
             fontsize=10, arrowprops=dict(arrowstyle='->', color='gray'))
ax1.annotate('Phase = 90°', xy=(25, 1), xytext=(60, 0.7),
             fontsize=10, arrowprops=dict(arrowstyle='->', color='gray'))

# Plot 2: Composite signal
ax2 = axes[1]
ax2.plot(t * 1000, composite, color=PRIMARY_GREEN, linewidth=2)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Composite Signal (5 + 10 + 20 Hz) — Phase is AMBIGUOUS', 
              fontsize=12, fontweight='bold', color=PRIMARY_RED)
ax2.set_xlim(0, 500)
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)

# Add warning annotation
ax2.text(250, 2, '❓ Which phase?\n5 Hz? 10 Hz? 20 Hz?', 
         ha='center', fontsize=11, color=PRIMARY_RED,
         bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_RED, alpha=0.8))

# Plot 3: Filtered signal
ax3 = axes[2]
ax3.plot(t * 1000, filtered_10hz, color=ACCENT_PURPLE, linewidth=2)
ax3.set_xlabel('Time (ms)', fontsize=11)
ax3.set_ylabel('Amplitude', fontsize=11)
ax3.set_title('Band-pass Filtered (8-12 Hz) — Phase Becomes Meaningful', 
              fontsize=12, fontweight='bold', color=PRIMARY_GREEN)
ax3.set_xlim(0, 500)
ax3.grid(True, alpha=0.3)
ax3.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)

# Add success annotation
ax3.text(400, 0.8, '✓ Now we can\nextract phase!', 
         ha='center', fontsize=11, color=PRIMARY_GREEN,
         bbox=dict(boxstyle='round', facecolor='white', edgecolor=PRIMARY_GREEN, alpha=0.8))

plt.suptitle('Visualization 1: The Phase Ambiguity Problem', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Key insight: We must filter to a narrow band before extracting phase.")
print("But HOW do we extract phase from the filtered signal? → Hilbert Transform!")

TypeError: bandpass_filter() got an unexpected keyword argument 'low_cutoff'

---

## 3. The Analytic Signal Concept

The key to extracting phase and amplitude is the **analytic signal**.

### Definition

For a real signal $x(t)$, the analytic signal $z(t)$ is:

$$z(t) = x(t) + i \cdot \hat{x}(t)$$

where:
- $x(t)$ is the original real signal
- $\hat{x}(t) = \mathcal{H}\{x(t)\}$ is the **Hilbert transform** of $x(t)$
- $i = \sqrt{-1}$

### What We Can Extract

From the analytic signal, we can compute:

| Quantity | Formula | Meaning |
|----------|---------|--------|
| **Instantaneous Amplitude** | $A(t) = |z(t)| = \sqrt{x^2 + \hat{x}^2}$ | The envelope (signal strength) |
| **Instantaneous Phase** | $\phi(t) = \arg(z(t)) = \text{atan2}(\hat{x}, x)$ | The oscillation phase |

### Geometric Interpretation: The Rotating Phasor

Think of the analytic signal as a **rotating vector** in the complex plane:
- The **real part** (x-axis) is the original signal
- The **imaginary part** (y-axis) is the Hilbert transform
- The **length** of the vector is the amplitude
- The **angle** of the vector is the phase

In [ ]:
# ============================================================================
# VISUALIZATION 2: The Analytic Signal as a Rotating Phasor
# ============================================================================

# Create a simple sine wave
fs = 250
duration = 0.5
t = np.arange(0, duration, 1/fs)
freq = 5  # Hz
signal_sin = np.sin(2 * np.pi * freq * t)

# Compute analytic signal
analytic = hilbert(signal_sin)
real_part = np.real(analytic)
imag_part = np.imag(analytic)
envelope = np.abs(analytic)
phase = np.angle(analytic)

# Create figure with phasor diagram and time series
fig = plt.figure(figsize=(14, 5))

# Left: Phasor diagram (complex plane)
ax1 = fig.add_subplot(1, 2, 1)

# Draw the unit circle
theta_circle = np.linspace(0, 2*np.pi, 100)
ax1.plot(np.cos(theta_circle), np.sin(theta_circle), 'k--', alpha=0.3, linewidth=1)

# Plot the trajectory of the analytic signal
ax1.plot(real_part, imag_part, color=PRIMARY_BLUE, linewidth=1.5, alpha=0.5, label='Trajectory')

# Show a few phasor snapshots
snapshot_indices = [0, 12, 25, 37, 50]
colors_snap = [PRIMARY_BLUE, PRIMARY_GREEN, ACCENT_GOLD, ACCENT_PURPLE, PRIMARY_RED]

for idx, color in zip(snapshot_indices, colors_snap):
    ax1.arrow(0, 0, real_part[idx]*0.95, imag_part[idx]*0.95, 
              head_width=0.08, head_length=0.05, fc=color, ec=color, linewidth=2)
    ax1.plot(real_part[idx], imag_part[idx], 'o', color=color, markersize=8)
    ax1.annotate(f't={t[idx]*1000:.0f}ms', xy=(real_part[idx], imag_part[idx]),
                 xytext=(real_part[idx]+0.15, imag_part[idx]+0.15), fontsize=9)

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-1.5, 1.5)
ax1.set_xlabel('Real Part: x(t)', fontsize=11)
ax1.set_ylabel('Imaginary Part: ĥ(t)', fontsize=11)
ax1.set_title('Phasor Diagram (Complex Plane)', fontsize=12, fontweight='bold')
ax1.set_aspect('equal')
ax1.axhline(y=0, color='gray', linestyle='-', linewidth=0.5)
ax1.axvline(x=0, color='gray', linestyle='-', linewidth=0.5)
ax1.grid(True, alpha=0.3)

# Add labels for amplitude and phase
idx_demo = 25
ax1.annotate('', xy=(real_part[idx_demo], imag_part[idx_demo]), xytext=(0, 0),
             arrowprops=dict(arrowstyle='<->', color=ACCENT_GOLD, lw=2))
ax1.text(0.3, 0.5, 'A(t) = |z(t)|', fontsize=10, color=ACCENT_GOLD, fontweight='bold')

# Right: Time series showing relationship
ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(t * 1000, real_part, color=PRIMARY_BLUE, linewidth=2, label='Real: x(t)')
ax2.plot(t * 1000, imag_part, color=SECONDARY_PINK, linewidth=2, label='Imag: ĥ(t)')
ax2.plot(t * 1000, envelope, color=ACCENT_GOLD, linewidth=2, linestyle='--', label='Envelope: |z(t)|')

# Mark snapshot times
for idx, color in zip(snapshot_indices, colors_snap):
    ax2.axvline(x=t[idx]*1000, color=color, linestyle=':', alpha=0.7)

ax2.set_xlabel('Time (ms)', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.set_title('Time Domain View', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('Visualization 2: The Analytic Signal — A Rotating Phasor', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("The phasor rotates counterclockwise at the signal frequency.")
print("Its length = amplitude, its angle = phase.")